<a href="https://colab.research.google.com/github/gunjanak/Pytorch_Course/blob/main/9_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path = '/content/drive/MyDrive/Colab Notebooks/Python_Course/PyTorch_Course/NTC_Stock.csv'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(path)

In [4]:
# Set the 'Name' column as the index
df.set_index('Date', inplace=True)

In [5]:
import numpy as np

In [6]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [7]:
# Prepare data for RNN
sequence_length = 5
X = []
y = []
for i in range(len(df) - sequence_length):
    X.append(df.iloc[i:i + sequence_length]['Close'].values)
    y.append(df.iloc[i + sequence_length]['Close'])

X = np.array(X)
y = np.array(y)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# Scale the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

In [10]:
# Split data into training and testing sets
split_index = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [11]:
import torch.nn as nn

In [12]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [14]:
# Instantiate the model
input_size = 1  # Size of each input sample
hidden_size = 64  # Number of features in the hidden state
output_size = 1  # Size of each output sample
model = LSTM(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [15]:
# Training the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor.unsqueeze(2))  # Unsqueezing to add the third dimension
    optimizer.zero_grad()
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/5], Loss: 1.2477
Epoch [2/5], Loss: 1.2269
Epoch [3/5], Loss: 1.2064
Epoch [4/5], Loss: 1.1861
Epoch [5/5], Loss: 1.1660


In [16]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor.unsqueeze(2))
    test_loss = criterion(outputs.squeeze(), y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 0.1356


In [17]:
# Save the trained model weights
torch.save(model.state_dict(), 'model_weights_LSTM.pth')

In [19]:
# Scale the 'Value' column using StandardScaler
scaler = StandardScaler()
df['Close_scaled'] = scaler.fit_transform(df[['Close']])

# Prepare data for feeding into the model
sequence_length = 5
X = []
for i in range(len(df) - sequence_length + 1):
    X.append(df['Close_scaled'].iloc[i:i + sequence_length].values)

# Convert to numpy array and then to PyTorch tensor
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(2)  # Add a third dimension



In [21]:
# Load the trained model weights
model.load_state_dict(torch.load('model_weights_LSTM.pth'))
model.eval()

LSTM(
  (lstm): LSTM(1, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [22]:
# Make predictions for the entire DataFrame
with torch.no_grad():
    forecast = []
    for i in range(len(X_tensor)):
        output = model(X_tensor[i].unsqueeze(0))  # Unsqueezing to add the batch dimension
        forecast.append(output.item())

In [ ]:
# Inverse transform the forecasted values to get back to the original scale
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

# Add the 'Forecast' column to the DataFrame
df['Forecast'] = np.nan
df['Forecast'].iloc[sequence_length-1:] = forecast

print(df)

In [24]:
df_final = df[['Close','Forecast']]

In [25]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Create trace for 'Close' data
trace_close = go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close')

# Create trace for 'Forecast' data
trace_forecast = go.Scatter(x=df.index, y=df['Forecast'], mode='lines', name='Forecast')

# Create layout
layout = go.Layout(title='Close vs Forecast',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Price'))

# Create figure with both traces
fig = go.Figure(data=[trace_close, trace_forecast], layout=layout)

# Plot the figure
pyo.iplot(fig)


#Multilayer LSTM

In [58]:
class MultiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MultiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=22, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(2, x.size(0), self.hidden_size).to(x.device)  # Changed 1 to 2 for num_layers
        c0 = torch.zeros(2, x.size(0), self.hidden_size).to(x.device)  # Changed 1 to 2 for num_layers

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [59]:
# Instantiate the model
input_size = 1  # Size of each input sample
hidden_size = 64  # Number of features in the hidden state
output_size = 1  # Size of each output sample
model_multi_lstm = MultiLSTM(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [60]:
# Training the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor.unsqueeze(2))  # Unsqueezing to add the third dimension
    optimizer.zero_grad()
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/5], Loss: 1.2184
Epoch [2/5], Loss: 1.2123
Epoch [3/5], Loss: 1.2063
Epoch [4/5], Loss: 1.2003
Epoch [5/5], Loss: 1.1942
